#### Master Thesis 2024 - Diksha Acharya

# 5 - Heat index classification for 3h and daily data
## Master Thesis 2024
---

5.1 - Imports and tables


In [1]:
# Import 
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
import os
from shapely.geometry import Point
from matplotlib.colors import ListedColormap, BoundaryNorm

In [3]:
# Indian states shapefile
shapefile_path = "Indian_States"
Indian_states = gpd.read_file(shapefile_path)

5.1.1 - T° and HI 3h

In [2]:
# table for 3h HI°C
file_HI_3h = "Heat_Index_C_3h_final_with_adjustments.nc"
HI_3h = xr.open_dataset(file_HI_3h)
HI_3h

<xarray.Dataset>
Dimensions:     (time: 245448, longitude: 120, latitude: 124)
Coordinates:
  * time        (time) datetime64[ns] 1940-01-01 ... 2023-12-31T21:00:00
  * longitude   (longitude) float32 68.0 68.25 68.5 68.75 ... 97.25 97.5 97.75
  * latitude    (latitude) float32 38.0 37.75 37.5 37.25 ... 8.0 7.75 7.5 7.25
Data variables:
    heat_index  (time, latitude, longitude) float32 ...

In [4]:
# table for 3h T°C
file_T_3h= "India_T_3h_masked_C.nc"
T_3h = xr.open_dataset(file_T_3h)
T_3h

<xarray.Dataset>
Dimensions:    (time: 245448, longitude: 120, latitude: 124)
Coordinates:
  * time       (time) datetime64[ns] 1940-01-01 ... 2023-12-31T21:00:00
  * longitude  (longitude) float32 68.0 68.25 68.5 68.75 ... 97.25 97.5 97.75
  * latitude   (latitude) float32 38.0 37.75 37.5 37.25 ... 8.0 7.75 7.5 7.25
Data variables:
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    CDI:          Climate Data Interface version 2.0.4 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed Oct 16 17:32:30 2024: cdo -b F32 mergetime 1940_India_t...
    CDO:          Climate Data Operators version 2.0.4 (https://mpimet.mpg.de...

5.1.2 - T° and HI daily 

In [5]:
# table for daily HI°C
file_HI_stat = "India_HI_adjustment_daily_stats.nc"
HI_daily = xr.open_dataset(file_HI_stat)
HI_daily

<xarray.Dataset>
Dimensions:       (longitude: 120, latitude: 124, time: 30681)
Coordinates:
  * longitude     (longitude) float32 68.0 68.25 68.5 68.75 ... 97.25 97.5 97.75
  * latitude      (latitude) float32 38.0 37.75 37.5 37.25 ... 8.0 7.75 7.5 7.25
  * time          (time) datetime64[ns] 1940-01-01 1940-01-02 ... 2023-12-31
Data variables:
    daily_max_HI  (time, latitude, longitude) float32 ...
    daily_min_HI  (time, latitude, longitude) float32 ...
    daily_avg_HI  (time, latitude, longitude) float32 ...

In [6]:
# table for daily T°C
file_T_stat = "India_T_daily_stats.nc"
T_daily = xr.open_dataset(file_T_stat)
T_daily

<xarray.Dataset>
Dimensions:         (longitude: 120, latitude: 124, time: 30681)
Coordinates:
  * longitude       (longitude) float32 68.0 68.25 68.5 ... 97.25 97.5 97.75
  * latitude        (latitude) float32 38.0 37.75 37.5 37.25 ... 7.75 7.5 7.25
  * time            (time) datetime64[ns] 1940-01-01 1940-01-02 ... 2023-12-31
Data variables:
    daily_max_temp  (time, latitude, longitude) float32 ...
    daily_min_temp  (time, latitude, longitude) float32 ...
    daily_avg_temp  (time, latitude, longitude) float32 ...

---
5.2 - HI categories 3h

    1- Extrem danger : 54°C >= (red)

    2- Danger : 41-54°C (orange-red)

    3- Extreme caution: 32-41°C (orange)

    4- Caution: 27-32°C (yellow)


5.2.1 - Binary matrice categories for HI 3h

In [7]:
# Extrem Danger: Values >= 54°C (red)
HI_extreme_danger_3h = xr.where(HI_3h['heat_index'] >= 54, 1, 0)

# Danger: Values between 41°C and 54°C (orange-red)
HI_danger_3h = xr.where((HI_3h['heat_index'] >= 41) & (HI_3h['heat_index'] < 54), 1, 0)

# Extreme Caution: Values between 32°C and 41°C (orange)
HI_extreme_caution_3h = xr.where((HI_3h['heat_index'] >= 32) & (HI_3h['heat_index'] < 41), 1, 0)

# Caution: Values between 27°C and 32°C (yellow)
HI_caution_3h = xr.where((HI_3h['heat_index'] >= 27) & (HI_3h['heat_index'] < 32), 1, 0)

In [8]:
# save matrices
HI_extreme_danger_3h.to_netcdf("HI_classes\HI_extreme_danger_3h.nc")
HI_danger_3h.to_netcdf("HI_classes\HI_danger_3h.nc")
HI_extreme_caution_3h.to_netcdf("HI_classes\HI_extreme_caution_3h.nc")
HI_caution_3h.to_netcdf("HI_classes\HI_caution_3h.nc")

In [9]:
file_HI_extreme_danger_3h = "HI_classes\HI_extreme_danger_3h.nc"
HI_extreme_danger_3h = xr.open_dataset(file_HI_extreme_danger_3h)
HI_extreme_danger_3h

<xarray.Dataset>
Dimensions:     (time: 245448, longitude: 120, latitude: 124)
Coordinates:
  * time        (time) datetime64[ns] 1940-01-01 ... 2023-12-31T21:00:00
  * longitude   (longitude) float32 68.0 68.25 68.5 68.75 ... 97.25 97.5 97.75
  * latitude    (latitude) float32 38.0 37.75 37.5 37.25 ... 8.0 7.75 7.5 7.25
Data variables:
    heat_index  (time, latitude, longitude) int32 ...

5.2.3 - Binary matrice categories for T° 3h

In [ ]:
# Extrem Danger: Values >= 54°C (red)
T_extreme_danger_3h = xr.where(T_3h['t2m'] >= 54, 1, 0)

# Danger: Values between 41°C and 54°C (orange-red)
T_danger_3h = xr.where((T_3h['t2m'] >= 41) & (T_3h['t2m'] < 54), 1, 0)

# Extreme Caution: Values between 32°C and 41°C (orange)
T_extreme_caution_3h = xr.where((T_3h['t2m'] >= 32) & (T_3h['t2m'] < 41), 1, 0)

# Caution: Values between 27°C and 32°C (yellow)
T_caution_3h = xr.where((T_3h['t2m'] >= 27) & (T_3h['t2m'] < 32), 1, 0)

In [ ]:
# save matrices
T_extreme_danger_3h.to_netcdf("T_classes\T_extreme_danger_3h.nc")
T_danger_3h.to_netcdf("T_classes\T_danger_3h.nc")
T_extreme_caution_3h.to_netcdf("T_classes\T_extreme_caution_3h.nc")
T_caution_3h.to_netcdf("T_classes\T_caution_3h.nc")

In [4]:
file_T_extreme_danger_3h = "T_classes\T_extreme_danger_3h.nc"
T_extreme_danger_3h = xr.open_dataset(file_T_extreme_danger_3h)
T_extreme_danger_3h

<xarray.Dataset>
Dimensions:    (time: 245448, longitude: 120, latitude: 124)
Coordinates:
  * time       (time) datetime64[ns] 1940-01-01 ... 2023-12-31T21:00:00
  * longitude  (longitude) float32 68.0 68.25 68.5 68.75 ... 97.25 97.5 97.75
  * latitude   (latitude) float32 38.0 37.75 37.5 37.25 ... 8.0 7.75 7.5 7.25
Data variables:
    t2m        (time, latitude, longitude) int32 ...

---
5.3 - HI categories daily max

    1- Extrem danger : 54°C >= (red)

    2- Danger : 41-54°C (orange-red)

    3- Extreme caution: 32-41°C (orange)

    4- Caution: 27-32°C (yellow)

5.3.1 - Binary matrice categories for HI daily max

In [10]:
# Extrem Danger: Values >= 54°C (red)
HI_extreme_danger_daily = xr.where(HI_daily['daily_max_HI'] >= 54, 1, 0)

# Danger: Values between 41°C and 54°C (orange-red)
HI_danger_daily = xr.where((HI_daily['daily_max_HI'] >= 41) & (HI_daily['daily_max_HI'] < 54), 1, 0)

# Extreme Caution: Values between 32°C and 41°C (orange)
HI_extreme_caution_daily = xr.where((HI_daily['daily_max_HI'] >= 32) & (HI_daily['daily_max_HI'] < 41), 1, 0)

# Caution: Values between 27°C and 32°C (yellow)
HI_caution_daily = xr.where((HI_daily['daily_max_HI'] >= 27) & (HI_daily['daily_max_HI'] < 32), 1, 0)

In [11]:
# save matrices
HI_extreme_danger_daily.to_netcdf("HI_classes\HI_extreme_danger_daily_max.nc")
HI_danger_daily.to_netcdf("HI_classes\HI_danger_daily_max.nc")
HI_extreme_caution_daily.to_netcdf("HI_classes\HI_extreme_caution_daily_max.nc")
HI_caution_daily.to_netcdf("HI_classes\HI_caution_daily_max.nc")

In [12]:
file_HI_extreme_danger_daily_max = "HI_classes\HI_extreme_danger_daily_max.nc"
HI_extreme_danger_daily_max = xr.open_dataset(file_HI_extreme_danger_daily_max)
HI_extreme_danger_daily_max

<xarray.Dataset>
Dimensions:       (longitude: 120, latitude: 124, time: 30681)
Coordinates:
  * longitude     (longitude) float32 68.0 68.25 68.5 68.75 ... 97.25 97.5 97.75
  * latitude      (latitude) float32 38.0 37.75 37.5 37.25 ... 8.0 7.75 7.5 7.25
  * time          (time) datetime64[ns] 1940-01-01 1940-01-02 ... 2023-12-31
Data variables:
    daily_max_HI  (time, latitude, longitude) int32 ...

5.3.2 - Binary matrice categories for T daily max

In [10]:
# Extrem Danger: Values >= 54°C (red)
T_extreme_danger_daily = xr.where(T_daily['daily_max_temp'] >= 54, 1, 0)

# Danger: Values between 41°C and 54°C (orange-red)
T_danger_daily = xr.where((T_daily['daily_max_temp'] >= 41) & (T_daily['daily_max_temp'] < 54), 1, 0)

# Extreme Caution: Values between 32°C and 41°C (orange)
T_extreme_caution_daily = xr.where((T_daily['daily_max_temp'] >= 32) & (T_daily['daily_max_temp'] < 41), 1, 0)

# Caution: Values between 27°C and 32°C (yellow)
T_caution_daily = xr.where((T_daily['daily_max_temp'] >= 27) & (T_daily['daily_max_temp'] < 32), 1, 0)

In [ ]:
# save matrices
T_extreme_danger_daily.to_netcdf("T_classes\T_extreme_danger_daily_max.nc")
T_danger_daily.to_netcdf("T_classes\T_danger_daily_max.nc")
T_extreme_caution_daily.to_netcdf("T_classes\T_extreme_caution_daily_max.nc")
T_caution_daily.to_netcdf("T_classes\T_caution_daily_max.nc")

In [8]:
file_T_extreme_danger_daily_max = "T_classes\T_extreme_danger_daily_max.nc"
T_extreme_danger_3h_daily_max = xr.open_dataset(file_T_extreme_danger_daily_max)
T_extreme_danger_3h_daily_max

<xarray.Dataset>
Dimensions:         (longitude: 120, latitude: 124, time: 30681)
Coordinates:
  * longitude       (longitude) float32 68.0 68.25 68.5 ... 97.25 97.5 97.75
  * latitude        (latitude) float32 38.0 37.75 37.5 37.25 ... 7.75 7.5 7.25
  * time            (time) datetime64[ns] 1940-01-01 1940-01-02 ... 2023-12-31
Data variables:
    daily_max_temp  (time, latitude, longitude) int32 ...